In [ ]:
# if you don’t want to install, just add the src folder to sys.path
import sys
sys.path.insert(0, "../src")

In [ ]:
from portablemsvc.manifest import get_vs_manifest

# turn off cache if you want a fresh download
vs_manifest = get_vs_manifest(cache=True)
print("Fetched VS manifest keys:", list(vs_manifest.keys())[:5])

In [ ]:
from portablemsvc.parse_manifest import parse_vs_manifest

parsed = parse_vs_manifest(
    vs_manifest,
    host="x64",            # your host arch
    targets=["x64"],       # your target archs
    msvc_version=None,     # None for latest
    sdk_version=None       # None for latest
)
print("Selected MSVC:", parsed["selected_msvc"])
print("Selected SDK:",  parsed["selected_sdk"]['version'])

In [ ]:
from portablemsvc.download_manifest import download_manifest_files

files_map = download_manifest_files(parsed)   # returns { filename → Path(cache) }
print("Downloaded manifest files:", files_map)

In [ ]:
from portablemsvc.parse_msi import parse_msi_for_cabs
from portablemsvc.download import download_files

sdk_info     = parsed["selected_sdk"]["package_info"]
cab_payloads = parse_msi_for_cabs(files_map, sdk_info)

cab_map = download_files(cab_payloads)

print("First Found CAB:", [key for key in cab_payloads.keys()][0])
print("First Downloaded CAB file:", [key for key in cab_map.keys()][0])

In [ ]:
from pathlib import Path
from portablemsvc.config import DATA_DIR
from portablemsvc.extract import extract_package_files

msvc_full = parsed["selected_msvc"]["full_version"]
sdk_ver   = parsed["selected_sdk"]["version"]

out_dir = Path(DATA_DIR) / f"msvc-{msvc_full}_sdk-{sdk_ver}"
all_files = {**files_map, **cab_map}

extracted = extract_package_files(all_files, out_dir)
print("Extraction complete.  Sample extracted paths:")
for p in list(extracted["msvc"])[:5]:
    print(" ", p)


In [ ]:
from portablemsvc.install import install_msvc_components

# This will perform the CRT wiring, msdia copy, cleanup, batch‐file creation
install_results = install_msvc_components(
    output_dir=out_dir,
    extracted_files=extracted,
    host="x64",            # your host arch
    targets=["x64"],       # your target archs
    msvc_version=msvc_full,
    sdk_version=sdk_ver
)

print("install_msvc_components returned:")
print(install_results)